In [ ]:
# start coding here    
import xarray as xr
import numpy as np
from fpcluster.read_trajectories import load_trajectories, read_trajectories
import dask
from dask.distributed import LocalCluster

In [ ]:
inpf = snakemake.input 
wildcards = snakemake.wildcards
paths_trajecs = inpf.paths_trajecs
paths_source_contrib = inpf.source_contrib_paths
time_stamps = []
ddep = []
threshold = snakemake.params.threshold
use_dask = snakemake.params.use_dask

In [ ]:
cluster = LocalCluster(n_workers=32, threads_per_worker=1)
client= dask.distributed.Client(cluster)

In [ ]:
for path in paths_source_contrib:
    if path.endswith('.zarr'):
        ds = xr.open_zarr(path)
    else:
        ds = xr.open_dataset(path)
#     if use_dask:
#     ds = ds.chunk(chunks={'time':30})
    ds = ds[ds.varName].sum(dim=['btime','lon','lat'])
    ddep.append(ds)
# ddep = xr.concat(ddep, com)    
# time_stamps.append(ds.time.values)

In [ ]:
ds = xr.concat(ddep, dim='time')

ds = ds.compute()

ds = ds.where(ds >= threshold,drop=True)

In [ ]:
ds_trajec = read_trajectories(paths_trajecs,kind=wildcards.kind)


In [ ]:
ds = ds.drop('height')

In [ ]:
# da = xr.DataArray(data=ddep, dims=['time'], coords={'time':time_stamps})
ds = ds[~ds.get_index("time").duplicated()] #avoid duplicate indices?
ds_trajec = ds_trajec.sel(time=ds.time[~ds.get_index("time").duplicated()])
ds_trajec = ds_trajec.assign({wildcards.kind:ds})
ds_trajec.attrs['varName'] = wildcards.kind
ds_trajec.to_netcdf(snakemake.output.outpath)